In [49]:
import os
import json
from dotenv import load_dotenv
import pandas as pd
from tqdm.notebook import tqdm

load_dotenv()  # take environment variables from .env.

True

In [50]:
# establish a connection to the MongoDB database
from pymongo import MongoClient

# connect to your Atlas cluster
client = MongoClient(os.environ["MONGODB_URI"])

In [51]:
# establish a connection to the PostgreSQL database
import psycopg2 as pg

conn = pg.connect(
    dbname=os.environ["POSTGRES_DB"],
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_HOST"]
)


# RDFlib

## Outline

 * Get data from the PostgreSQL DB
 * Populate graph with triples


In [52]:
from rdflib import Graph, Literal, URIRef
from rdflib.namespace import RDF, RDFS, XSD, FOAF, OWL
import regex as re


In [53]:
# craete a graph
g = Graph()

# create a namespace object
fdo = "http://vu-game-ontology.com/ontology/"
fdr = "http://vu-game-ontology.com/resource/"

# bind the namespace to the graph
g.bind("fdo", fdo)
g.bind("fdr", fdr)


### Get Metacritic data

In [54]:
df_mc = pd.read_sql("SELECT * FROM metacritic_pages", conn)

/tmp/ipykernel_1697/1308410040.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_mc = pd.read_sql("SELECT * FROM metacritic_pages", conn)


In [55]:
df_mc.head()

,object_id,title,score,platforms,release_date,developers,publisher,genres,fandom_game_url
0,6718cddc17c581114a9d20cf,Super Mario Galaxy 2,97.0,[Wii],2010-05-23,[Nintendo EAD Tokyo],Nintendo,[3D Platformer],None
1,6718cedb17c581114a9d20d0,The Legend of Zelda: Breath of the Wild,97.0,"[Wii U, Nintendo Switch]",2017-03-03,[Nintendo],Nintendo,[Open-World Action],None
2,6718cedb17c581114a9d20d1,Red Dead Redemption 2,97.0,"[Xbox One, PlayStation 4, PC]",2018-10-26,[Rockstar Games],Rockstar Games,[Open-World Action],None
3,6718cedb17c581114a9d20d3,Super Mario Odyssey,97.0,[Nintendo Switch],2017-10-27,[Nintendo],Nintendo,[3D Platformer],None
4,6718cedc17c581114a9d20d5,Mass Effect 2,96.0,"[PC, Xbox 360, PlayStation 3]",2010-01-26,[BioWare],Electronic Arts,[Western RPG],None


In [56]:
# df_mc.release_date = pd.to_datetime(df_mc.release_date)

In [57]:
pd.isna(df_mc.release_date).sum()

2

In [58]:
g.add((URIRef(fdo + 'hasMetacriticScore'), RDFS.subPropertyOf, URIRef(fdo + 'hasReceptionScore')))
g.add((URIRef(fdo + 'hasMetacriticScore'), RDF.type, OWL.DatatypeProperty))
g.add((URIRef(fdo + 'releaseDate'), RDF.type, OWL.DatatypeProperty))
g.add((URIRef(fdo + 'hasGenre'), RDF.type, OWL.ObjectProperty))
g.add((URIRef(fdo + 'hasPlatform'), RDF.type, OWL.ObjectProperty))
g.add((URIRef(fdo + 'developedBy'), RDF.type, OWL.ObjectProperty))
g.add((URIRef(fdo + 'publishedBy'), RDF.type, OWL.ObjectProperty))

# Classes
g.add((URIRef(fdo + "Game"), RDF.type, OWL.Class))
g.add((URIRef(fdo + "Genre"), RDF.type, OWL.Class))
g.add((URIRef(fdo + "Publisher"), RDF.type, OWL.Class))
g.add((URIRef(fdo + "Developer"), RDF.type, OWL.Class))
g.add((URIRef(fdo + "Platform"), RDF.type, OWL.Class))



for row in tqdm(df_mc.to_dict(orient="records")):
    # create a URIRef object for the current game
    game = URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', row["title"]))
    # add a type triple to the graph
    g.add((game, RDF.type, URIRef(fdo + "Game")))

    # add a title (label) triple to the graph
    g.add((game, RDFS.label, Literal(row["title"], lang="en")))

    # add a year triple to the graph
    if not pd.isna(row["release_date"]):
        g.add((game, URIRef(fdo + "releaseDate"), Literal(row["release_date"], datatype=XSD.date)))

    # add a metascore triple to the graph, if it is not NaN
    if not pd.isna(row["score"]):
        g.add((game, URIRef(fdo + "hasMetacriticScore"), Literal(row["score"], datatype=XSD.integer)))

    for genre in row["genres"]:
        # add a genre triple to the graph
        g.add((game, URIRef(fdo + "hasGenre"), URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', genre))))
        g.add((URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', genre)), RDF.type, URIRef(fdo + "Genre")))

    for platform in row["platforms"]:
        # add a platform triple to the graph
        g.add((game, URIRef(fdo + "hasPlatform"), URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', platform))))
        g.add((URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', platform)), RDF.type, URIRef(fdo + "Platform")))

    if row["developers"] is not None:
        for developer in row["developers"]:
            # add a developer triple to the graph
            g.add((game, URIRef(fdo + "developedBy"), URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', developer))))
            g.add((URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', developer)), RDF.type, URIRef(fdo + "Developer")))

    # add publisher triple to the graph
    g.add((game, URIRef(fdo + "publishedBy"), URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', row["publisher"]))))
    g.add((URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', row["publisher"])), RDF.type, URIRef(fdo + "Publisher")))

    if row['fandom_game_url'] is not None:
        # add a URL triple to the graph
        g.add((game, OWL.sameAs, URIRef(row['fandom_game_url'])))
        g.add((game, OWL.sameAs, URIRef(fdr + row['fandom_game_url'].split('/')[-2].split('.')[0])))





  0%|          | 0/7826 [00:00<?, ?it/s]

### Get triples from fandom for page properties

In [59]:
df_classes = pd.read_sql_query(
    """select distinct class from fandom_entities_clean
    union
    select distinct superclass from fandom_entities_clean
    union
    select distinct subclass from fandom_entities_clean
    ;
    """,
    conn)
unique_classes = list(df_classes['class'].unique())

/tmp/ipykernel_1697/523897747.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_classes = pd.read_sql_query(


In [60]:
df_top_classes = pd.read_sql_query(
    """select distinct class from fandom_entities_clean
    union
    select distinct superclass from fandom_entities_clean
    ;
    """,
    conn)
top_lvl_classes = list(df_top_classes['class'].unique())

/tmp/ipykernel_1697/2112334253.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_top_classes = pd.read_sql_query(


In [61]:
# get data from postgres
df_triples = pd.read_sql_query(
    """select
       p.object_id,
       game_name,
       game_url,
       page_name,
       page_url,
       name as subject_name,
       class,
       subclass,
       description,
       target_entity as object_name,
       object_class,
       property_name,
       subject_class,
       property_id,
       property_type,
       property_description

from fandom_pages p
join fandom_entities_clean e on p.object_id = e.object_id
join fandom_triples_clean t on p.object_id = t.object_id
    ;""",
      conn)

/tmp/ipykernel_1697/4097690141.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_triples = pd.read_sql_query(


In [62]:
df_triples.shape

(75431, 16)

In [63]:
df_triples.head()

,object_id,game_name,game_url,page_name,page_url,subject_name,class,subclass,description,object_name,object_class,property_name,subject_class,property_id,property_type,property_description
0,6713a3a88f64cb721b2f5bfc,endlessspace2,https://endless-space-2.fandom.com/,The_Remnant,https://endless-space-2.fandom.com/wiki/The_Re...,Remnant,Faction,Civilization,The Remnant is a faction in Endless Space 2 kn...,Concentrated Constructors I (50/70 Chance): -1...,Trait,hasAssimilationTrait,Faction,4010,object,Traits that affect how the faction assimilates...
1,67139f9e8f64cb721b2f3eec,bravelydefault,https://bravelydefault.fandom.com/,Adam_Holograd,https://bravelydefault.fandom.com/wiki/Adam_Ho...,Adam Holograd,Character,LordCommander,Adam Holograd is a character from the game Bra...,Silver,Color,hasHairColor,Character,3529,data,"Specifies the hair color of the character, whi..."
2,6713a3a98f64cb721b2f5bff,endlessspace2,https://endless-space-2.fandom.com/,The_Sowers,https://endless-space-2.fandom.com/wiki/The_So...,Sowers,GameFaction,AlienFaction,The Sowers are a religious alien faction known...,"+2, -0.5 Depletion Points per turn on Planets",PopulationBonus,hasPopulationBonus,Faction,1218,object,The bonuses granted to the faction based on po...
3,6713a3a98f64cb721b2f5c01,endlessspace2,https://endless-space-2.fandom.com/,The_Tikanan,https://endless-space-2.fandom.com/wiki/The_Ti...,Tikanan,Faction,MilitaristFaction,Tikanan is a militarist faction known for its ...,Swarming Infantry (50/70 Chance): 17.5% more I...,Trait,hasAssimilationTrait,Faction,4010,object,Traits that affect how the faction assimilates...
4,6713a3a98f64cb721b2f5c02,endlessspace2,https://endless-space-2.fandom.com/,The_Xirmisala,https://endless-space-2.fandom.com/wiki/The_Xi...,Xirmisala,Civilization,Faction,Xirmisala is a faction in Endless Space 2 know...,"Academy Ally: +10% Contributions to Academy, E...",Trait,hasAssimilationTrait,Faction,4010,object,Traits that affect how the faction assimilates...


In [64]:
skipped = 0

g.add((URIRef(fdo + "ObjectClass"), RDF.type, OWL.Class))

# iterate over the rows of the dataframe
for row in tqdm(df_triples.to_dict(orient="records")):
    # create a URIRef object for the entity

    subject = URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', row["page_url"].split('wiki/')[-1]))

    # check for none values
    if row["object_name"].lower() in ["none", "unknown", "n/a", ""]:
        skipped += 1
        continue

    # create an object name for the target entity
    object_name = re.sub('[^A-Za-z0-9_]+', '', row["object_name"])

    # check if the triple is a class definition
    # if row["object_class"].lower() == "class":
    if row["property_name"] == "hasType":
        # add a triple for the entity class
        g.add((subject, RDF.type, URIRef(fdo + object_name)))
        if object_name not in unique_classes:
            g.add((URIRef(fdo + object_name), RDFS.subClassOf, URIRef(fdo + row["subclass"])))
            g.add((URIRef(fdo + object_name), RDFS.label, Literal(row["object_name"], lang="en")))
            g.add((URIRef(fdo + object_name), RDF.type, OWL.Class))

    else:
        object = URIRef(fdr + object_name)

        property = URIRef(fdo + row["property_name"])
    
        # # check is subpropertyof exists:
        # if row["subpropertyof"] is not None and row["subpropertyof"] != row["property_name"]:
        #     g.add((property, RDFS.subPropertyOf, URIRef(fdo + row["subpropertyof"])))

        if row["property_type"] == "object":
            # add a triple for the target entity class
            g.add((object, RDF.type, URIRef(fdo + row["object_class"])))
            g.add((URIRef(fdo + row["object_class"]), RDF.type, OWL.Class))
            object_class = re.sub('[^A-Za-z0-9_]+', '', row["object_class"])
            if object_class not in unique_classes:
                g.add((URIRef(fdo + object_class), RDFS.subClassOf, URIRef(fdo + "ObjectClass")))

            # add a triple for the target entity name
            g.add((object, RDFS.label, Literal(row["object_name"], lang="en")))

            # add a triple for the entity
            g.add((subject, property, object))
            g.add((property, RDF.type, OWL.ObjectProperty))

        else:
            # add a triple for the entity
            try:
                g.add((subject, property, Literal(int(row["object_name"]), datatype=XSD.integer)))
            except ValueError:
                g.add((subject, property, Literal(row["object_name"], lang="en")))

            g.add((property, RDF.type, OWL.DatatypeProperty))

        # add a triple for property description
        g.add((property, RDFS.comment, Literal(row["property_description"], lang="en")))

print(f"Skipped {skipped} triples")


  0%|          | 0/75431 [00:00<?, ?it/s]

Skipped 1822 triples


#### Add triples for properties

In [65]:
df_props = pd.read_sql_query(
    """select * from fandom_ontology_properties;""",
    conn)

/tmp/ipykernel_1697/4048350202.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_props = pd.read_sql_query(


In [66]:
df_props.head()

,id,property_name,description,subject_class,object_class,property_type,subpropertyof,original_property_ids,cluster_id
0,1253,foundInLocation,Indicates the location where the item can be f...,Item,Location,object,None,"[7532, 7593, 7625, 7829, 7936, 7977, 11511, 12...",209
1,1254,foundInCharacter,Indicates the character from whom the item can...,Item,Character,object,None,"[7549, 9755, 26774, 26809, 26817, 26838, 26847...",209
2,1255,foundInEvent,Indicates the event during which the item can ...,Item,Event,object,None,"[26775, 26818, 26839, 42143, 26848]",209
3,1256,foundInGame,Indicates the game in which the item can be fo...,Item,Game,object,None,"[16192, 16245, 5615, 5627, 5647]",209
4,1257,foundInCulture,Indicates the culture or group associated with...,Event,Culture,object,None,"[16210, 16214, 16218, 16222, 16225, 16228, 162...",209


In [67]:
for row in tqdm(df_props.to_dict(orient="records")):
    # create a URIRef object for the property
    property = URIRef(fdo + row["property_name"])

    # check if subpropertyof exists
    if row["subpropertyof"] is not None and row["subpropertyof"] != row["property_name"]:
        g.add((property, RDFS.subPropertyOf, URIRef(fdo + row["subpropertyof"])))

  0%|          | 0/4751 [00:00<?, ?it/s]

#### Add subject classes

In [68]:
df_subjects = pd.read_sql_query(
    """select distinct
       p.object_id,
       game_name,
       game_url,
       page_name,
       page_url,
       name as subject_name,
       class,
       subclass,
       superclass,
       description

from fandom_pages p
join fandom_entities_clean e on p.object_id = e.object_id
    ;""",
    conn)

/tmp/ipykernel_1697/574267213.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_subjects = pd.read_sql_query(


In [69]:
# get a subset based on unique subjects

df_subjects = df_subjects.drop_duplicates(subset=["game_name", "game_url", "page_name", "page_url", "subject_name", "class", "superclass"])
df_subjects = df_subjects[["game_name", "game_url", "page_name", "page_url", "subject_name", "class", "subclass", "description", "superclass"]]

In [70]:
df_subjects.head()

,game_name,game_url,page_name,page_url,subject_name,class,subclass,description,superclass
0,iamsetsuna,https://iamsetsuna.fandom.com/,Aurorean_Tiger,https://iamsetsuna.fandom.com/wiki/Aurorean_Tiger,Aurorean Tiger,Monster,Beast,Aurorean Tiger is a monster known for its stre...,Creature
1,wildstar,https://wildstar.fandom.com/,AMP_Power_Upgrade,https://wildstar.fandom.com/wiki/AMP_Power_Upg...,AMP Power Upgrade,Item,UpgradeItem,An item used to unlock additional points of AM...,Entity
2,tevi,https://tevi.fandom.com/,Burnt_Dessert,https://tevi.fandom.com/wiki/Burnt_Dessert,Burnt Dessert,Item,Consumable,Burnt Dessert is a crafting item used within t...,Entity
3,battleblocktheater,https://battleblocktheater.fandom.com/,Gordon,https://battleblocktheater.fandom.com/wiki/Gordon,Gordon,Character,NPC,Gordon is a shipmate and friend character in B...,None
4,vvvvvv,https://vvvvvv.fandom.com/,DSS_Souleye,https://vvvvvv.fandom.com/wiki/DSS_Souleye,DSS Souleye,Item,Trinket,The DSS Souleye is a trinket featured in the g...,Entity


In [71]:
for row in tqdm(df_subjects.to_dict(orient="records")):
    # create a URIRef object for the entity
    subject = URIRef(fdr + re.sub('[^A-Za-z0-9_]+', '', row["page_url"].split('wiki/')[-1]))

    # add a triple for the entity class
    g.add((subject, RDF.type, URIRef(fdo + row["subclass"])))

    # add subclass
    if row["subclass"] not in top_lvl_classes:
        g.add((URIRef(fdo + row["subclass"]), RDFS.subClassOf, URIRef(fdo + row["class"])))
        g.add((URIRef(fdo + row["class"]), RDF.type, OWL.Class))
        g.add((URIRef(fdo + row["subclass"]), RDF.type, OWL.Class))

    # # compare the subject class with the class 
    # if row["class"] != row["subject_class"]:
    #         g.add((URIRef(fdo + row["class"]), RDFS.subClassOf, URIRef(fdo + row["subject_class"])))
    #         g.add((URIRef(fdo + row["subject_class"]), RDF.type, OWL.Class))

    if row["superclass"] is not None:
        g.add((URIRef(fdo + row["class"]), RDFS.subClassOf, URIRef(fdo + row["superclass"])))
        g.add((URIRef(fdo + row["superclass"]), RDF.type, OWL.Class))

    # add a triple for the entity name
    g.add((subject, RDFS.label, Literal(row["subject_name"], lang="en")))

    # add a triple for the entity description
    g.add((subject, RDFS.comment, Literal(row["description"], lang="en")))

    g.add((subject, OWL.sameAs, URIRef(row["page_url"])))



  0%|          | 0/12945 [00:00<?, ?it/s]

In [72]:
g.serialize("fandom_temp.ttl", format="turtle")

<Graph identifier=N7b7cf83e99784024941863e3d5aa8c7c (<class 'rdflib.graph.Graph'>)>